# Dropout과 Dead Relu는 어떤 차이점이 있을까?

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class 

# https://discuss.pytorch.org/t/how-to-fix-define-the-initialization-weights-seed/20156